In [1]:
import pandas as pd
import os

In [14]:
def obtener_gastos_indirectos_directos(cc_a_obtener, dfs_cubo):
    historico_cubo = pd.DataFrame()
    for fecha_df, df in dfs_cubo.items():
        df = df.set_index('Insumos / Centro Costos')
        gasto_total = df.loc['Total Directos', cc_a_obtener].to_frame()
        gasto_total['Total Indirectos'] = df.loc['Total Indirectos', cc_a_obtener]
        gasto_total = gasto_total.transpose()
        gasto_total['Fecha'] = pd.to_datetime(fecha_df, format='%Y_%m')
        historico_cubo = pd.concat([historico_cubo, gasto_total])

    historico_cubo = historico_cubo.reset_index()

    return historico_cubo

In [57]:
ruta_archivos = [os.path.join('historicos_cubo', ruta) for ruta in os.listdir('historicos_cubo')]
ruta_archivos = sorted(ruta_archivos, key=lambda x: int(x.split('_')[3]))

In [58]:
dfs_cubo = {'_'.join(ruta.split('_')[4: 6]): pd.read_excel(ruta) for ruta in ruta_archivos}

In [59]:
cc_a_obtener = ['66-HOSPITALIZACIÓN MEDICINA INTERNA', '90-HOSPITALIZACIÓN QUIRÚRGICA',
                '102-HOSPITALIZACIÓN CARDIOVASCULAR', '464-QUIRÓFANOS CARDIOVASCULAR',
                '484-QUIRÓFANOS TORACICA', '166-UNIDAD DE CUIDADOS INTENSIVOS',
                '195-UNIDAD DE TRATAMIENTO INTENSIVO ADULTO']

In [60]:
historico_directo_indirecto = obtener_gastos_indirectos_directos(cc_a_obtener=cc_a_obtener,
                                                                 dfs_cubo=dfs_cubo)

In [61]:
cubo_2022 = {llave: valor for llave, valor in dfs_cubo.items() if '2022' in llave}

In [73]:
suma = cubo_2022['2022_1']
suma['Insumos / Centro Costos'] = suma['Insumos / Centro Costos'] + '_'
acumulado = 0
for fecha_df, df in list(cubo_2022.items())[1:]:
    valor_x = df.iloc[0, 1]
    print(f'Uno de los valores es {valor_x}')
    acumulado += valor_x
    suma = suma.add(df)

suma['Insumos / Centro Costos'] = df['Insumos / Centro Costos']
print(f'El acumulado es {acumulado}')

Uno de los valores es 86594560
Uno de los valores es 126368297
Uno de los valores es 90456855
Uno de los valores es 82640829
Uno de los valores es 144221548
Uno de los valores es 83541214
Uno de los valores es 85410200
Uno de los valores es 126346103
El acumulado es 825579606


In [67]:
suma.to_csv('cubo_acumulado_2022.csv', sep=';', encoding='latin-1', index=False)

In [70]:
suma

In [15]:
ruta_archivos = [os.path.join('historicos_producciones', ruta) for ruta in os.listdir('historicos_producciones')]
ruta_archivos = sorted(ruta_archivos)


In [16]:
dfs_produccion = {ruta.split()[-2]: pd.read_excel(ruta) for ruta in ruta_archivos}

In [22]:
columnas = {"Unnamed: 2": "ENERO",
            "Unnamed: 3": "FEBRERO",
            "Unnamed: 4": "MARZO",
            "Unnamed: 5": "ABRIL",
            "Unnamed: 6": "MAYO",
            "Unnamed: 7": "JUNIO",
            "Unnamed: 8": "JULIO",
            "Unnamed: 9": "AGOSTO",
            "Unnamed: 10": "SEPTIEMBRE",
            "Unnamed: 11": "OCTUBRE",
            "Unnamed: 12": "NOVIEMBRE",
            "Unnamed: 13": "DICIEMBRE"}

In [26]:
dfs_produccion['2021'] = dfs_produccion['2021'].rename(columns=columnas)
dfs_produccion['2022'] = dfs_produccion['2022'].rename(columns=columnas)

In [64]:
tavi_2021 = dfs_produccion['2021'].loc[62]['ENERO':'DICIEMBRE']
tavi_2022 = dfs_produccion['2022'].loc[62]['ENERO':'DICIEMBRE']

cantidad_tavis = pd.concat([tavi_2021, tavi_2022]).reset_index().drop(columns=['index'])

In [70]:
gastos_tavi['Cantidad_TAVIs'] = cantidad_tavis.astype('Int32')
gastos_tavi = gastos_tavi.replace(0, 0)

In [71]:
gastos_tavi['Gasto_Indirecto_promedio'] = gastos_tavi['Gasto_Indirecto'].divide(gastos_tavi['Cantidad_TAVIs'])

In [73]:
gastos_tavi.to_excel('tavi_indirectos_historicos.xlsx', index=False)